# <font color='darkred'>Tese de Mestrado</font>
## Modelling Clustering

In [ ]:
# Importação de Bibliotecas
import pandas as pd
import numpy as np
import Levenshtein
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_excel('df_final.xlsx')
df.head()

In [ ]:
colunas = list(df.columns)
print(colunas)

### <font color='darkred'>Tratamento</font>
#### Identificação dos Valores Omissos

In [ ]:
total_omissos = df.isna().sum().sum()
print("\nNúmero total de valores omissos no DataFrame:", total_omissos)

In [ ]:
# Substituição dos valores omissos por NA
df.replace('', np.nan, inplace=True)

## <font color='darkred'>Clustering</font>
### Variáveis Utilizadas no Clustering:
- Tipo de Contrato;
- Salário (rganho);
- Cae (caem1l);
- Profissão (prof_3d);
- Sexo;
- Nº de Anos no Privado.
- Trocas de Empresa

#### Tipo de Contrato:
- Criação de uma coluna que resulta da copilação do Tipo de Contrato de cada Trabalhador ao longo dos anos.

In [ ]:
df_trab = df.copy()

In [ ]:
df_trab.columns

In [ ]:
def tratamento_contrato(contrato):
    contrato = str(contrato)  
    if contrato == 'Não Trabalhou':
        return '0'
    else:
        return contrato  

def criacao_trajetoria(row):
    anos = [str(year)[-2:] for year in range(2009, 2020)]
    trajetoria = ''
    for ano in anos:
        col = f'tipo_contr1_{ano}'
        tipos = str(row[col])  
        trajetoria += tratamento_contrato(tipos)
    return trajetoria  #

# Criação de uma coluna com a trajetória ao longo dos anos
df_trab['trajetoria_contratos'] = df_trab.apply(criacao_trajetoria, axis=1)

df_trab.head()

In [ ]:
trajetorias_unicas = df_trab['trajetoria_contratos'].unique()
n_trajetorias_unicas = len(trajetorias_unicas)

print("\nNúmero total de trajetórias únicas:", n_trajetorias_unicas)

**Distância de Levenshtein**
- Para calcular a próximidade desta variável utilizar-se-á a distância de Levenshtein que mede o número mínimo de operações necessárias para transformar uma string em outra.

No exemplo que se segue a trajetória de referência é '1111'
| trajetoria_contratos | distancia |
|----------------------|---------------------|
| 1111                 | 0                   |
| 0001                 | 3                   |
| 0101                 | 2                   |
| 0000                 | 4                   |
| 0010                 | 3                   |

In [ ]:
# Top 10 de trajetórias mais frequentes
valor_contagem = df_trab['trajetoria_contratos'].value_counts()

top_10_valores = valor_contagem.head(10)
top_10_valores

In [ ]:
trajetoria_referencia = '11111111111'
print(f"Trajetória de referência: {trajetoria_referencia}")

In [ ]:
def calcular_distancia(trajetoria, referencia):
    return Levenshtein.distance(trajetoria, referencia)

# Calcular a distância de todas as trajetórias únicas para a trajetória de referência
distancias = [calcular_distancia(traj, trajetoria_referencia) for traj in trajetorias_unicas]

df_distancias = pd.DataFrame({'trajetoria': trajetorias_unicas, 'dist_tipocontr': distancias})
df_distancias.head()

In [ ]:
df_merged = df_distancias.merge(df_trab, left_on='trajetoria', right_on='trajetoria_contratos')

df_merged.drop(columns=['trajetoria'], inplace=True)

cols = [col for col in df_merged.columns if col not in ['dist_tipocontr']]
cols += ['dist_tipocontr']
df_merged = df_merged[cols]

df_merged.head()

### Salário:
- Crescimento médio anual para cada trabalhador.

O cálculo a seguir demonstra como será feito o cálculo do Crescimento Médio Anual para o trabalhador com `ntrab` 322655233:

| Ano  | Ganho (R$)   | Crescimento (%)  |
|------|--------------|-------------------|
| 2009 | 1566,7       | -                 |
| 2010 | 957,047      | -38,9%            |
| 2011 | 902,602      | -5,7%             |
| 2012 | 862,69       | -4,4%             |
| 2013 | 797,81       | -7,5%             |
| 2014 | 800,032      | +0,3%             |
| 2015 | 796,155      | -0,5%             |
| 2016 | 791,34       | -0,6%             |
| 2017 | 780,66       | -1,3%             |
| 2018 | 772,97       | -1,0%             |
| 2019 | 1059,26      | +37,1%            |

**Fórmula de Crescimento Percentual Anual:**

Para calcular o crescimento percentual anual entre dois anos, use a fórmula:

`Crescimento (%) = ((Ganho_ano2 - Ganho_ano1) / Ganho_ano1) * 100`

**Crescimento Médio Anual:**

Para calcular o crescimento médio anual, somamos todos os crescimentos percentuais e dividimos pelo número de períodos (número de variações). No caso, temos 10 períodos (de 2009 a 2018).

Crescimento Total = -38,9\% - 5,7\% - 4,4\% - 7,5\% + 0,3\% - 0,5\% - 0,6\% - 1,3\% - 1,0\% + 37,1\% = 22,2\% \ 10 = -2,22\%

In [ ]:
for year_suffix in range(9, 20):
    col_name = f'rganho_h_{year_suffix:02d}'
    df_merged[col_name] = df_merged[col_name].astype(str)

# Função para calcular o crescimento salarial médio anual
def calcular_crescimento_medio(row):
    ganhos = []
    anos = list(range(2009, 2020))  
    
    for ano in anos:
        year_suffix = str(ano)[-2:]  
        col_name = f'rganho_h_{year_suffix}'
        ganho = row[col_name]
        
        # Substituir "Não Trabalhou" por NaN temporariamente dentro da função
        if ganho == "Não Trabalhou":
            ganho = np.nan
        
        if pd.notna(ganho) and pd.to_numeric(ganho, errors='coerce') > 0:  # Considerar apenas ganhos positivos
            ganhos.append((ano, pd.to_numeric(ganho, errors='coerce')))
    
    if len(ganhos) < 2:
        return 0.0  
    
    crescimento_total = 0
    for i in range(len(ganhos) - 1):
        ano1, ganho1 = ganhos[i]
        ano2, ganho2 = ganhos[i + 1]
        
        if ganho1 == 0:
            crescimento = 0  # Evitar divisão por zero
        else:
            crescimento = (ganho2 - ganho1) / ganho1
        
        crescimento_total += crescimento
    
    crescimento_medio_anual = crescimento_total / (len(ganhos) - 1)
    return crescimento_medio_anual

df_merged['crescimento_medio_anual'] = round(df_merged.apply(calcular_crescimento_medio, axis=1),2)
df_merged.head()

In [ ]:
# Calcular o valor mínimo e máximo da variável 'crescimento_medio_anual_zscore'
min_crescimento = df_merged['crescimento_medio_anual'].min()
max_crescimento = df_merged['crescimento_medio_anual'].max()

print(f'Valor Mínimo: {min_crescimento}')
print(f'Valor Máximo: {max_crescimento}')

#### Código de Atividade Económica:
- Cálculo do CAE mais frequente ao longo dos anos.

In [ ]:
caem1l_cols = [col for col in df_merged.columns if col.startswith('caem1l')]

# Função para calcular o CAE mais frequente, substituindo "Não Trabalhou" por NaN 
def cae_mais_frequente(row):
    valores = [val for val in row[caem1l_cols] if pd.notna(val) and val != "Não Trabalhou"]
    if valores:
        return pd.Series(valores).mode(dropna=True).iloc[0]
    else:
        return np.nan

# Adicionar a coluna com o CAE mais frequente
df_merged['cae_mais_frequente'] = df_merged.apply(cae_mais_frequente, axis=1)
df_merged.head()

In [ ]:
df_merged['cae_mais_frequente'].value_counts()

#### Profissão:
- Cálculo da prof_3d mais frequente ao longo dos anos.

In [ ]:
prof_3d_cols = [col for col in df_merged.columns if col.startswith('prof_3d')]

# Função para calcular o valor mais frequente, substituindo "Não Trabalhou" por NaN
def valor_mais_frequente(row):
    valores = [val for val in row[prof_3d_cols] if pd.notna(val) and val != "Não Trabalhou"]
    # Calcula o valor mais frequente
    if valores:
        return pd.Series(valores).mode(dropna=True).iloc[0]
    else:
        return np.nan

# Adicionar a coluna com o valor mais frequente
df_merged['prof_3d_mais_frequente'] = df_merged.apply(valor_mais_frequente, axis=1)
df_merged.head()

In [ ]:
df_merged['prof_3d_mais_frequente'].value_counts()

#### Habilitações Literárias:
- Criação de uma coluna boleana que indica se um trabalhador aumentou o seu grau académico ou não. Para tal, serão comparados as variáveis habil_09 e habil_19, considerando que valores como "Não Trabalhou" na coluna habil_19 serão substituídos pela última variável habil que contém valor.

In [ ]:
# Função para encontrar o valor de grau acadêmico mais recente válido
def encontrar_grau_valido(row):
    for ano in range(19, 8, -1): 
        col = f'habil1_{ano:02d}'
        valor = row[col]
        if valor not in ["Não trabalhou", "9"]:
            try:
                return int(valor)  # Tentar converter o valor para inteiro
            except ValueError:
                continue
    return np.nan

# Aplicar a função para criar uma nova coluna com o valor de grau acadêmico válido mais recente
df_merged['último_habil1'] = df_merged.apply(encontrar_grau_valido, axis=1)

# Função para verificar se o grau acadêmico aumentou
def grau_academico_aumentou(row):
    try:
        habil1_09 = int(row['habil1_09'])  # Converter habil1_09 para inteiro
    except ValueError:
        return 0
    
    if pd.isna(row['último_habil1']):
        return 0 
    
    return 1 if row['último_habil1'] > habil1_09 else 0

# Criar a nova coluna booleana com base na comparação
df_merged['aumentou_habil'] = df_merged.apply(grau_academico_aumentou, axis=1)

df_merged.drop(columns=['último_habil1'], inplace=True)

df_merged.head()

In [ ]:
df_merged['aumentou_habil'].value_counts()

#### Número de Anos no Privado:
- Criação de uma coluna que resulta da análise de todas as colunas caeml1 (por não conter NAS ao longo dos 11 anos). Posteriormente, analisar-se-á o número de anos que um dado trabalhador estave no setor privado. 

In [ ]:
def contar_anos_privado(row):
    anos_trabalhados = 0
    for col in df_merged.columns:
        if col.startswith('caem1l_'):
            if row[col] != 'Não Trabalhou':
                anos_trabalhados += 1
    return anos_trabalhados

df_merged['anos_privado'] = df_merged.apply(contar_anos_privado, axis=1)
df_merged.head()

In [ ]:
df_merged['anos_privado'].value_counts()

#### Número de Trocas de Empresa:

Criação de uma coluna que resulta da contabilização do número de trocas de empresa.
- Se o valor atual e o valor seguinte são inteiros e são diferentes, conta como uma troca.
- Se o valor atual é um inteiro e o valor seguinte é "Não Trabalhou", conta como uma troca.
- Se o valor atual é "Não Trabalhou" e o valor seguinte é NaN (ou seja, não há valor), conta como uma troca. A troca é contabilizada usando o último valor não NaN válido antes de "Não Trabalhou".
- Se o valor atual é um inteiro e o valor seguinte é NaN, não conta como uma troca.
- Se ambos os valores (atual e seguinte) são NaN ou "Não Trabalhou", não contabiliza como troca.
- Quando um valor é NaN, ele é substituído pelo último valor não NaN válido para a comparação.

Na tabela abaixo estão 2 exemplos que refletem como este cálculo será feito

| nuemp_09 | nuemp_10 | nuemp_11 | nuemp_12 | nuemp_13       | nuemp_14       | nuemp_15       | nuemp_16       | nuemp_17 | nuemp_18 | nuemp_19 | trocas_empresa |
|----------|----------|----------|----------|----------------|----------------|----------------|----------------|----------|----------|----------|----------------|
| 234      | 234      | 211      | 212      | Não Trabalhou  | Não Trabalhou  | None           | None           | 245      | 245      | 246      | 5              |
| 234      | 234      | 211      | 212      | None           | None           | None           | None           | 245      | 245      | 246      | 4              |
 |
 |
|


In [ ]:
# Definir a função para contar trocas de empresa
def contar_trocas_empresa(row):
    trocas = 0
    ultimo_valor = None 
    
    for ano in range(9, 19):  # De 2009 a 2018
        col_atual = f'nuemp_{ano:02d}'
        col_seguinte = f'nuemp_{ano+1:02d}'
        
        valor_atual = row.get(col_atual, pd.NA)
        valor_seguinte = row.get(col_seguinte, pd.NA)
        
        if pd.notna(valor_atual) and valor_atual != "Não Trabalhou":
            ultimo_valor = valor_atual
        
        # Substituir NaN com o último valor não NaN
        valor_atual = ultimo_valor if pd.isna(valor_atual) else valor_atual
        valor_seguinte = ultimo_valor if pd.isna(valor_seguinte) else valor_seguinte
        
        # Se ambos os valores são NaN ou "Não Trabalhou", não contabiliza troca
        if pd.isna(valor_atual) and pd.isna(valor_seguinte):
            continue
        if valor_atual == "Não Trabalhou" and valor_seguinte == "Não Trabalhou":
            continue
        
        # Se o valor atual for um inteiro e o seguinte for "Não Trabalhou"
        if isinstance(valor_atual, int):
            if valor_seguinte == "Não Trabalhou":
                trocas += 1
                continue
        
        # Se o valor atual for "Não Trabalhou" e o seguinte for NaN
        if valor_atual == "Não Trabalhou" and pd.isna(valor_seguinte):
            trocas += 1
            continue
        
        # Se o valor atual for um inteiro e o seguinte for NaN
        if isinstance(valor_atual, int) and pd.isna(valor_seguinte):
            continue
        
        # Se ambos os valores são inteiros diferentes entre si
        if isinstance(valor_atual, int) and isinstance(valor_seguinte, int):
            if valor_atual != valor_seguinte:
                trocas += 1
    
    return trocas

df_merged['trocas_empresa'] = df_merged.apply(contar_trocas_empresa, axis=1)
df_merged.head()

In [ ]:
df_merged.tail()

In [ ]:
df_merged['trocas_empresa'].value_counts()

In [ ]:
media_trocas_emprego = round(df_merged['trocas_empresa'].mean(),2)
print(f"A média de trocas de emprego é: {media_trocas_emprego}")

In [ ]:
# df_clustering_final = df_merged.iloc[:, [1, 2] + list(range(-8, 0))]
# df_clustering_final.to_excel('df_clustering_final.xlsx', index=False)

### <font color='darkred'>Clustering I </font>
#### <font color='black'>Variáveis Mais Frequentes<font>

In [ ]:
df_merged.head()

In [ ]:
filtro_2 = df_original_clustering[df_original_clustering['trajetoria_contratos'].astype(str).str.startswith('2')]
top_5_cae_2 = filtro_2['cae_mais_frequente'].value_counts().head(5)

filtro_3 = df_original_clustering[df_original_clustering['trajetoria_contratos'].astype(str).str.startswith('3')]
top_5_cae_3 = filtro_3['cae_mais_frequente'].value_counts().head(5)

# Filtrar quando 'trajetoria_contratos' começa com '1'
filtro_1 = df_original_clustering[df_original_clustering['trajetoria_contratos'].astype(str).str.startswith('1')]
top_5_cae_1 = filtro_1['cae_mais_frequente'].value_counts().head(5)

print("Top 5 de 'cae_mais_frequente' quando 'trajetoria_contratos' começa com 2:")
print(top_5_cae_2)

print("Top 5 de 'cae_mais_frequente' quando 'trajetoria_contratos' começa com 3:")
print(top_5_cae_3)

print("\nTop 5 de 'cae_mais_frequente' quando 'trajetoria_contratos' começa com 1:")
print(top_5_cae_1)

In [ ]:
filtro_2 = df_original_clustering[df_original_clustering['trajetoria_contratos'].astype(str).str.startswith('2')]
top_5_cae_2 = filtro_2['prof_3d_mais_frequente'].value_counts().head(5)

filtro_3 = df_original_clustering[df_original_clustering['trajetoria_contratos'].astype(str).str.startswith('3')]
top_5_cae_3 = filtro_3['prof_3d_mais_frequente'].value_counts().head(5)

# Filtrar quando 'trajetoria_contratos' começa com '1'
filtro_1 = df_original_clustering[df_original_clustering['trajetoria_contratos'].astype(str).str.startswith('1')]
top_5_cae_1 = filtro_1['prof_3d_mais_frequente'].value_counts().head(5)

print("Top 5 de 'prof_3d_mais_frequente' quando 'trajetoria_contratos' começa com 2:")
print(top_5_cae_2)

print("Top 5 de 'prof_3d_mais_frequente' quando 'trajetoria_contratos' começa com 3:")
print(top_5_cae_3)

print("\nTop 5 de 'prof_3d_mais_frequente' quando 'trajetoria_contratos' começa com 1:")
print(top_5_cae_1)


In [ ]:
filtro_zeros = df_original_clustering[df_original_clustering['trajetoria_contratos'].str[-10:] == '0000000000']

# Contar os casos que começam com '1', '2', ou '3'
contagem_inicial = filtro_zeros['trajetoria_contratos'].str[0].value_counts()

print("Contagem de 'trajetoria_contratos' que terminam com 10 zeros e começam com 1, 2 ou 3:")
print(contagem_inicial)

In [ ]:
filtro_zeros.shape

In [ ]:
variaveis_clustering = [
    'sexo',
    'trajetoria_contratos',
    'dist_tipocontr',
    'crescimento_medio_anual',
    'cae_mais_frequente',
    'prof_3d_mais_frequente',
    'aumentou_habil',
    'anos_privado',
    'trocas_empresa'
]

df_original_clustering = df_merged[variaveis_clustering].copy()
df_original_clustering.head()

In [ ]:
# df_original_clustering.to_excel('df_clustering.xlsx', index=False)

In [ ]:
df_clustering = df_original_clustering.drop(columns=['sexo', 'trajetoria_contratos'])

In [ ]:
# Normalizar os dados de 0 a 1
scaler = MinMaxScaler()
df_clustering_normalized = df_clustering.copy()
df_clustering_normalized[df_clustering.columns] = scaler.fit_transform(df_clustering)

In [ ]:
X = df_clustering_normalized.values
n_clusters_range = range(1, 11)  
inertia = []

# Calculo da inércia para cada número de clusters
for n_clusters in n_clusters_range:
    kmeans = KMeans(n_clusters=n_clusters, random_state=0)
    kmeans.fit(X)
    inertia.append(kmeans.inertia_)

plt.figure(figsize=(8, 5))
plt.plot(n_clusters_range, inertia, marker='o', color='#0D3B66')
plt.xlabel('Número de Clusters')
plt.ylabel('Inércia')
plt.title('Método do Cotovelo para Determinação do Número Ideal de Clusters')
plt.xticks(n_clusters_range)
plt.grid(True)
plt.show()

In [ ]:
X = df_clustering_normalized.values

n_clusters_range = range(2, 11)  
silhouette_scores = []

# Calculo do índice da silhueta para cada número de clusters
for n_clusters in n_clusters_range:
    kmeans = KMeans(n_clusters=n_clusters, random_state=0)
    cluster_labels = kmeans.fit_predict(X)
    silhouette_avg = silhouette_score(X, cluster_labels)
    silhouette_scores.append(silhouette_avg)

max_silhouette_score = max(silhouette_scores)
optimal_clusters = n_clusters_range[silhouette_scores.index(max_silhouette_score)]

print(f"O índice de silhueta mais alto é {max_silhouette_score:.2f}, obtido com {optimal_clusters} clusters.")


plt.figure(figsize=(2, 4))
plt.plot(n_clusters_range, silhouette_scores, marker='o')
plt.xlabel('Número de Clusters')
plt.ylabel('Índice da Silhueta')
plt.title('Método do Índice da Silhueta para Determinação do Número Ideal de Clusters')
plt.xticks(n_clusters_range)
plt.grid(True)
plt.text(optimal_clusters, max_silhouette_score, f"Máximo: {max_silhouette_score:.2f}", 
         horizontalalignment='left', fontsize=10, color='red')

In [ ]:
kmeans = KMeans(n_clusters=3, random_state=42)
clusters = kmeans.fit_predict(df_clustering_normalized)

df_original_clustering['cluster'] = clusters
df_original_clustering.head()

#### Número de Trabalhadores por Cluster 

In [ ]:
n_clusters = df_original_clustering['cluster'].value_counts().sort_index()
n_clusters

#### Resumo das Características de Cada Cluster
- Usando a moda nas variáveis trajetoria_contratos, cae_mais_frequente e prof_3d_mais_frequente;

In [ ]:
clusters = df_original_clustering['cluster'].unique()

resultados = []

# Calcular moda e média para cada cluster
for cluster in clusters:
    cluster_data = df_original_clustering[df_original_clustering['cluster'] == cluster]
    
    moda = cluster_data[['trajetoria_contratos', 'cae_mais_frequente',
                         'prof_3d_mais_frequente']].mode().iloc[0]
    
    media_sexo = cluster_data['sexo'].mean()
    media_dist_tipocontr = cluster_data['dist_tipocontr'].mean()
    media_aumentou_habil = cluster_data['aumentou_habil'].mean()
    media_cae_frequente = cluster_data['cae_mais_frequente'].mean()
    media_anos_privado = cluster_data['anos_privado'].mean()
    media_trocas_empresa = cluster_data['trocas_empresa'].mean()
    media_crescimento_medio_anual = cluster_data['crescimento_medio_anual'].mean()
    
    resultados.append({
        'Cluster': cluster,
        'Sexo': media_sexo,
        'trajetoria_contratos': moda['trajetoria_contratos'],
        'Dist_Tipocontr': media_dist_tipocontr,
        'CAE_Mais_Frequente': moda['cae_mais_frequente'],
        'Prof_3D_Mais_Frequente': moda['prof_3d_mais_frequente'],
        'Aumentou_Habil': media_aumentou_habil,
        'Anos_Privado': media_anos_privado,
        'Trocas_Empresa': media_trocas_empresa,
        'Crescimento_Salarial': media_crescimento_medio_anual
    })

resultados_df = pd.DataFrame(resultados)

resultados_df = resultados_df.sort_values(by='Cluster').reset_index(drop=True)

resultados_df[['Sexo', 'Dist_Tipocontr', 'Aumentou_Habil', 'Anos_Privado', 'Trocas_Empresa', 'Crescimento_Salarial']] = \
resultados_df[['Sexo', 'Dist_Tipocontr', 'Aumentou_Habil', 'Anos_Privado', 'Trocas_Empresa', 'Crescimento_Salarial']].round(2)

resultados_df

- Top5 Prfissões por Cluster

In [ ]:
# Cluster 0
cluster = df_original_clustering[df_original_clustering['cluster'] == 0]
frequencia_prof_3d = cluster['prof_3d_mais_frequente'].value_counts()
frequencia_prof_3d.head(5)

In [ ]:
# Cluster 1
cluster = df_original_clustering[df_original_clustering['cluster'] == 1]
frequencia_prof_3d = cluster['prof_3d_mais_frequente'].value_counts()
frequencia_prof_3d.head(5)

In [ ]:
# Cluster 2
cluster = df_original_clustering[df_original_clustering['cluster'] == 2]
frequencia_prof_3d = cluster['prof_3d_mais_frequente'].value_counts()
frequencia_prof_3d.head(5)

In [ ]:
# Cluster 0
cluster = df_original_clustering[df_original_clustering['cluster'] == 0]
frequencia_prof_3d = cluster['cae_mais_frequente'].value_counts()
frequencia_prof_3d.head(5)

In [ ]:
# Cluster 1
cluster = df_original_clustering[df_original_clustering['cluster'] == 1]
frequencia_prof_3d = cluster['cae_mais_frequente'].value_counts()
frequencia_prof_3d.head(5)

In [ ]:
# Cluster 2
cluster = df_original_clustering[df_original_clustering['cluster'] == 2]
frequencia_prof_3d = cluster['cae_mais_frequente'].value_counts()
frequencia_prof_3d.head(5)

### <font color='darkred'>Clustering II </font>
#### <font color='black'>Variáveis Mais Frequentes s/ Habil</font>
- De acordo com o resumo das características de cada cluster foi possível verificar que a variável "aumentou_habil" está a ter uma presença muito significativa no modelo, estando os clusters a serem formados perfeitamente em função desta variável.
- Por isso, esta variável será retirada do modelo, com o intuito de ver a performance do mesmo.

In [ ]:
df_original_clustering_2 = df_merged[variaveis_clustering].copy()
df_original_clustering_2.head()

In [ ]:
df_clustering_2 = df_original_clustering_2.drop(columns=['sexo', 'trajetoria_contratos', 'aumentou_habil'])

In [ ]:
# Normalizar os dados de 0 a 1
scaler = MinMaxScaler()
df_clustering_normalized_2 = df_clustering_2.copy()
df_clustering_normalized_2[df_clustering_2.columns] = scaler.fit_transform(df_clustering_2)

In [ ]:
X = df_clustering_normalized_2.values

n_clusters_range = range(1, 11) 
inertia = []

# Calcular a inércia para cada número de clusters
for n_clusters in n_clusters_range:
    kmeans = KMeans(n_clusters=n_clusters, random_state=0)
    kmeans.fit(X)
    inertia.append(kmeans.inertia_)

plt.figure(figsize=(8, 5))
plt.plot(n_clusters_range, inertia, marker='o', color='#0D3B66')
plt.xlabel('Número de Clusters')
plt.ylabel('Inércia')
plt.title('Método do Cotovelo para Determinação do Número Ideal de Clusters')
plt.xticks(n_clusters_range)
plt.grid(True)
plt.show()

In [ ]:
X = df_clustering_normalized_2.values

n_clusters_range = range(2, 11)  
silhouette_scores = []

# Calculo do índice da silhueta para cada número de clusters
for n_clusters in n_clusters_range:
    kmeans = KMeans(n_clusters=n_clusters, random_state=0)
    cluster_labels = kmeans.fit_predict(X)
    silhouette_avg = silhouette_score(X, cluster_labels)
    silhouette_scores.append(silhouette_avg)

max_silhouette_score = max(silhouette_scores)
optimal_clusters = n_clusters_range[silhouette_scores.index(max_silhouette_score)]

print(f"O índice de silhueta mais alto é {max_silhouette_score:.2f}, obtido com {optimal_clusters} clusters.")

plt.figure(figsize=(8, 6))
plt.plot(n_clusters_range, silhouette_scores, marker='o')
plt.xlabel('Número de Clusters')
plt.ylabel('Índice da Silhueta')
plt.title('Método do Índice da Silhueta para Determinação do Número Ideal de Clusters')
plt.xticks(n_clusters_range)
plt.grid(True)
plt.text(optimal_clusters, max_silhouette_score, f"Máximo: {max_silhouette_score:.2f}", 
         horizontalalignment='left', fontsize=10, color='red')

In [ ]:
kmeans = KMeans(n_clusters=3, random_state=42)
clusters = kmeans.fit_predict(df_clustering_normalized_2)

df_original_clustering_2['cluster'] = clusters

df_original_clustering_2.head()

#### Número de Trabalhadores por Cluster 

In [ ]:
n_clusters = df_original_clustering_2['cluster'].value_counts().sort_index()
n_clusters

#### Resumo das Características de Cada Cluster
- Usando a moda nas variáveis trajetoria_contratos, cae_mais_frequente e prof_3d_mais_frequente;

In [ ]:
clusters = df_original_clustering_2['cluster'].unique()

resultados = []

# Calcular moda e média para cada cluster
for cluster in clusters:
    cluster_data = df_original_clustering_2[df_original_clustering_2['cluster'] == cluster]
    
    moda = cluster_data[['trajetoria_contratos', 'cae_mais_frequente',
                         'prof_3d_mais_frequente']].mode().iloc[0]
    
    media_sexo = cluster_data['sexo'].mean()
    media_dist_tipocontr = cluster_data['dist_tipocontr'].mean()
    media_aumentou_habil = cluster_data['aumentou_habil'].mean()
    media_anos_privado = cluster_data['anos_privado'].mean()
    media_trocas_empresa = cluster_data['trocas_empresa'].mean()
    media_crescimento_medio_anual_zscore = cluster_data['crescimento_medio_anual'].mean()
    
    resultados.append({
        'Cluster': cluster,
        'Sexo': media_sexo,
        'trajetoria_contratos': moda['trajetoria_contratos'],
        'Dist_Tipocontr': media_dist_tipocontr,
        'CAE_Mais_Frequente': moda['cae_mais_frequente'],
        'Prof_3D_Mais_Frequente': moda['prof_3d_mais_frequente'],
        'Aumentou_Habil': media_aumentou_habil,
        'Anos_Privado': media_anos_privado,
        'Trocas_Empresa': media_trocas_empresa,
        'Crescimento_Salarial': media_crescimento_medio_anual_zscore
    })

resultados_df = pd.DataFrame(resultados)

resultados_df = resultados_df.sort_values(by='Cluster').reset_index(drop=True)

resultados_df[['Sexo', 'Dist_Tipocontr', 'Aumentou_Habil', 'Anos_Privado', 'Trocas_Empresa', 'Crescimento_Salarial']] = \
resultados_df[['Sexo', 'Dist_Tipocontr', 'Aumentou_Habil', 'Anos_Privado', 'Trocas_Empresa', 'Crescimento_Salarial']].round(2)

resultados_df

In [ ]:
# Cluster 0
cluster = df_original_clustering_2[df_original_clustering_2['cluster'] == 0]
frequencia_prof_3d = cluster['prof_3d_mais_frequente'].value_counts()
frequencia_prof_3d.head(5)

In [ ]:
# Cluster 1
cluster = df_original_clustering_2[df_original_clustering_2['cluster'] == 1]
frequencia_prof_3d = cluster['prof_3d_mais_frequente'].value_counts()
frequencia_prof_3d.head(5)

In [ ]:
# Cluster 2
cluster = df_original_clustering_2[df_original_clustering_2['cluster'] == 2]
frequencia_prof_3d = cluster['prof_3d_mais_frequente'].value_counts()
frequencia_prof_3d.head(5)

### <font color='darkred'>Clustering III </font>
#### <font color='black'>Variáveis Início da Trajetória (2009)</font>

In [ ]:
variaveis_clustering_3 = [
    'sexo',
    'trajetoria_contratos',
    'dist_tipocontr',
    'crescimento_medio_anual',
    'caem1l_09',
    'prof_3d_09',
    'habil1_09',
    'anos_privado',
    'trocas_empresa'
]

df_original_clustering_3= df_merged[variaveis_clustering_3].copy()
df_original_clustering_3.head()

In [ ]:
df_clustering_3 = df_original_clustering_3.drop(columns=['sexo', 'trajetoria_contratos'])

In [ ]:
# Normalizar os dados de 0 a 1
scaler = MinMaxScaler()
df_clustering_normalized_3 = df_clustering_3.copy()
df_clustering_normalized_3[df_clustering_3.columns] = scaler.fit_transform(df_clustering_3)

In [ ]:
X = df_clustering_normalized_3.values
n_clusters_range = range(1, 11)  
inertia = []

# Calculo da inércia para cada número de clusters
for n_clusters in n_clusters_range:
    kmeans = KMeans(n_clusters=n_clusters, random_state=0)
    kmeans.fit(X)
    inertia.append(kmeans.inertia_)

plt.figure(figsize=(8, 5))
plt.plot(n_clusters_range, inertia, marker='o', color='#0D3B66')
plt.xlabel('Número de Clusters')
plt.ylabel('Inércia')
plt.title('Método do Cotovelo para Determinação do Número Ideal de Clusters')
plt.xticks(n_clusters_range)
plt.grid(True)
plt.show()

In [ ]:
X = df_clustering_normalized_3.values

n_clusters_range = range(2, 11)  
silhouette_scores = []

# Calculo do índice da silhueta para cada número de clusters
for n_clusters in n_clusters_range:
    kmeans = KMeans(n_clusters=n_clusters, random_state=0)
    cluster_labels = kmeans.fit_predict(X)
    silhouette_avg = silhouette_score(X, cluster_labels)
    silhouette_scores.append(silhouette_avg)

max_silhouette_score = max(silhouette_scores)
optimal_clusters = n_clusters_range[silhouette_scores.index(max_silhouette_score)]

print(f"O índice de silhueta mais alto é {max_silhouette_score:.2f}, obtido com {optimal_clusters} clusters.")


plt.figure(figsize=(8, 6))
plt.plot(n_clusters_range, silhouette_scores, marker='o')
plt.xlabel('Número de Clusters')
plt.ylabel('Índice da Silhueta')
plt.title('Método do Índice da Silhueta para Determinação do Número Ideal de Clusters')
plt.xticks(n_clusters_range)
plt.grid(True)
plt.text(optimal_clusters, max_silhouette_score, f"Máximo: {max_silhouette_score:.2f}", 
         horizontalalignment='left', fontsize=10, color='red')

In [ ]:
kmeans = KMeans(n_clusters=3, random_state=42)
clusters = kmeans.fit_predict(df_clustering_normalized_3)

df_original_clustering_3['cluster'] = clusters
df_original_clustering_3.head()

#### Número de Trabalhadores por Cluster 

In [ ]:
n_clusters = df_original_clustering_3['cluster'].value_counts().sort_index()
n_clusters

#### Resumo das Características de Cada Cluster
- Usando a moda nas variáveis trajetoria_contratos, cae_mais_frequente e prof_3d_mais_frequente;

In [ ]:
clusters = df_original_clustering_3['cluster'].unique()
resultados = []

# Calcular moda e média para cada cluster
for cluster in clusters:
    cluster_data = df_original_clustering_3[df_original_clustering_3['cluster'] == cluster]
    
    moda = cluster_data[['trajetoria_contratos','caem1l_09',
                         'prof_3d_09']].mode().iloc[0]
    
    media_sexo = cluster_data['sexo'].mean()
    media_dist_tipocontr = cluster_data['dist_tipocontr'].mean()
    media_habil = cluster_data['habil1_09'].mean()
    media_cae = cluster_data['caem1l_09'].mean()
    media_anos_privado = cluster_data['anos_privado'].mean()
    media_trocas_empresa = cluster_data['trocas_empresa'].mean()
    media_crescimento_medio_anual = cluster_data['crescimento_medio_anual'].mean()
    
    resultados.append({
        'Cluster': cluster,
        'Sexo': media_sexo,
        'trajetoria_contratos': moda['trajetoria_contratos'],
        'Dist_Tipocontr': media_dist_tipocontr,
        'CAE_09': moda['caem1l_09'],
        'Prof_3D_09': moda['prof_3d_09'],
        'Habil_09': media_habil,
        'Anos_Privado': media_anos_privado,
        'Trocas_Empresa': media_trocas_empresa,
        'Crescimento_Salarial': media_crescimento_medio_anual
    })

resultados_df = pd.DataFrame(resultados)

resultados_df = resultados_df.sort_values(by='Cluster').reset_index(drop=True)

resultados_df[['Sexo', 'Dist_Tipocontr', 'CAE_09', 'Habil_09', 'Anos_Privado', 'Trocas_Empresa', 'Crescimento_Salarial']] = \
resultados_df[['Sexo', 'Dist_Tipocontr', 'CAE_09', 'Habil_09', 'Anos_Privado', 'Trocas_Empresa', 'Crescimento_Salarial']].round(2)

resultados_df

In [ ]:
# Cluster 0
cluster = df_original_clustering_3[df_original_clustering_3['cluster'] == 0]
frequencia_prof_3d = cluster['prof_3d_09'].value_counts()
frequencia_prof_3d.head(5)

In [ ]:
# Cluster 1
cluster = df_original_clustering_3[df_original_clustering_3['cluster'] == 1]
frequencia_prof_3d = cluster['prof_3d_09'].value_counts()
frequencia_prof_3d.head(5)

In [ ]:
# Cluster 2
cluster = df_original_clustering_3[df_original_clustering_3['cluster'] == 2]
frequencia_prof_3d = cluster['prof_3d_09'].value_counts()
frequencia_prof_3d.head(5)

### <font color='darkred'>Clustering IV </font>
#### <font color='black'>Remoção das variáveis que não estão balanceada</font>

In [ ]:
variaveis_clustering_4 = [
    'sexo',
    'trajetoria_contratos',
    'dist_tipocontr',
    'crescimento_medio_anual',
    'anos_privado',
    'trocas_empresa'
]

df_original_clustering_4 = df_merged[variaveis_clustering_4].copy()
df_original_clustering_4.head()

In [ ]:
df_clustering_4 = df_original_clustering_4.drop(columns=['sexo', 'trajetoria_contratos'])

In [ ]:
# Normalizar os dados de 0 a 1
scaler = MinMaxScaler()
df_clustering_normalized_4 = df_clustering_4.copy()
df_clustering_normalized_4[df_clustering_4.columns] = scaler.fit_transform(df_clustering_4)

In [ ]:
X = df_clustering_normalized_4.values
n_clusters_range = range(1, 11)
inertia = []

for n_clusters in n_clusters_range:
    kmeans = KMeans(n_clusters=n_clusters, random_state=0)
    kmeans.fit(X)
    inertia.append(kmeans.inertia_)

plt.figure(figsize=(8, 5))
plt.plot(n_clusters_range, inertia, marker='o', color='#0D3B66')
plt.xlabel('Número de Clusters')
plt.ylabel('Inércia')
plt.title('Método do Cotovelo para Determinação do Número Ideal de Clusters')
plt.xticks(n_clusters_range)
plt.grid(True)
plt.show()

In [ ]:
# Aplicar K-Means com o número ideal de clusters
kmeans = KMeans(n_clusters=3, random_state=42)
clusters = kmeans.fit_predict(df_clustering_normalized_4)
df_original_clustering_4['cluster'] = clusters

df_original_clustering_4.head()

In [ ]:
n_clusters = df_original_clustering_4['cluster'].value_counts().sort_index()
n_clusters

In [ ]:
# K-Means com o número ideal de clusters
kmeans = KMeans(n_clusters=3, random_state=42)
clusters = kmeans.fit_predict(df_clustering_normalized_4)
df_original_clustering_4['cluster'] = clusters

df_original_clustering_4.head()

In [ ]:
clusters = df_original_clustering_4['cluster'].unique()
resultados = []

# Calcular moda e média para cada cluster
for cluster in clusters:
    cluster_data = df_original_clustering_4[df_original_clustering_4['cluster'] == cluster]
    
    moda = cluster_data[['trajetoria_contratos']].mode().iloc[0]
    
    media_sexo = cluster_data['sexo'].mean()
    media_dist_tipocontr = cluster_data['dist_tipocontr'].mean()
    media_anos_privado = cluster_data['anos_privado'].mean()
    media_trocas_empresa = cluster_data['trocas_empresa'].mean()
    media_crescimento_medio_anual = cluster_data['crescimento_medio_anual'].mean()
    
    resultados.append({
        'Cluster': cluster,
        'Sexo': media_sexo,
        'trajetoria_contratos': moda['trajetoria_contratos'],
        'Dist_Tipocontr': media_dist_tipocontr,
        'Anos_Privado': media_anos_privado,
        'Trocas_Empresa': media_trocas_empresa,
        'Crescimento_Salarial': media_crescimento_medio_anual
    })

resultados_df = pd.DataFrame(resultados)
resultados_df = resultados_df.sort_values(by='Cluster').reset_index(drop=True)

resultados_df[['Sexo', 'Dist_Tipocontr', 'Anos_Privado', 'Trocas_Empresa', 'Crescimento_Salarial']] = \
resultados_df[['Sexo', 'Dist_Tipocontr', 'Anos_Privado', 'Trocas_Empresa', 'Crescimento_Salarial']].round(2)

resultados_df

### <font color='darkred'>Clustering IV </font>
#### <font color='black'>Variáveis Mais Frequentes + habil_09</font>

In [ ]:
variaveis_clustering_4 = [
    'sexo',
    'trajetoria_contratos',
    'dist_tipocontr',
    'crescimento_medio_anual',
    'cae_mais_frequente',
    'prof_3d_mais_frequente',
    'habil1_09',
    'anos_privado',
    'trocas_empresa'
]

df_original_clustering_4 = df_merged[variaveis_clustering_4].copy()
df_original_clustering_4.head()

In [ ]:
df_clustering_4 = df_original_clustering_4.drop(columns=['sexo', 'trajetoria_contratos'])

In [ ]:
# Normalizar os dados de 0 a 1
scaler = MinMaxScaler()
df_clustering_normalized_4 = df_clustering_4.copy()
df_clustering_normalized_4[df_clustering_4.columns] = scaler.fit_transform(df_clustering_4)

In [ ]:
X = df_clustering_normalized_4.values

n_clusters_range = range(1, 11) 
inertia = []

# Calcular a inércia para cada número de clusters
for n_clusters in n_clusters_range:
    kmeans = KMeans(n_clusters=n_clusters, random_state=0)
    kmeans.fit(X)
    inertia.append(kmeans.inertia_)

plt.figure(figsize=(8, 5))
plt.plot(n_clusters_range, inertia, marker='o', color='#0D3B66')
plt.xlabel('Número de Clusters')
plt.ylabel('Inércia')
plt.title('Método do Cotovelo para Determinação do Número Ideal de Clusters')
plt.xticks(n_clusters_range)
plt.grid(True)
plt.show()

In [ ]:
X = df_clustering_normalized_4.values

n_clusters_range = range(2, 11)  
silhouette_scores = []

# Calculo do índice da silhueta para cada número de clusters
for n_clusters in n_clusters_range:
    kmeans = KMeans(n_clusters=n_clusters, random_state=0)
    cluster_labels = kmeans.fit_predict(X)
    silhouette_avg = silhouette_score(X, cluster_labels)
    silhouette_scores.append(silhouette_avg)

max_silhouette_score = max(silhouette_scores)
optimal_clusters = n_clusters_range[silhouette_scores.index(max_silhouette_score)]

print(f"O índice de silhueta mais alto é {max_silhouette_score:.2f}, obtido com {optimal_clusters} clusters.")


plt.figure(figsize=(8, 6))
plt.plot(n_clusters_range, silhouette_scores, marker='o')
plt.xlabel('Número de Clusters')
plt.ylabel('Índice da Silhueta')
plt.title('Método do Índice da Silhueta para Determinação do Número Ideal de Clusters')
plt.xticks(n_clusters_range)
plt.grid(True)
plt.text(optimal_clusters, max_silhouette_score, f"Máximo: {max_silhouette_score:.2f}", 
         horizontalalignment='left', fontsize=10, color='red')

In [ ]:
kmeans = KMeans(n_clusters=3, random_state=42)
clusters = kmeans.fit_predict(df_clustering_normalized_4)

df_original_clustering_4['cluster'] = clusters

df_original_clustering_4.head()

#### Número de Trabalhadores por Cluster 

In [ ]:
n_clusters = df_original_clustering_4['cluster'].value_counts().sort_index()
n_clusters

#### Resumo das Características de Cada Cluster
- Usando a moda nas variáveis trajetoria_contratos, cae_mais_frequente e prof_3d_mais_frequente;

In [ ]:
clusters = df_original_clustering_4['cluster'].unique()
resultados = []

# Calcular moda e média para cada cluster
for cluster in clusters:
    cluster_data = df_original_clustering_4[df_original_clustering_4['cluster'] == cluster]
    
    moda = cluster_data[['trajetoria_contratos', 'cae_mais_frequente',
                         'prof_3d_mais_frequente']].mode().iloc[0]
    
    media_sexo = cluster_data['sexo'].mean()
    media_dist_tipocontr = cluster_data['dist_tipocontr'].mean()
    media_aumentou_habil = cluster_data['habil1_09'].mean()
    media_anos_privado = cluster_data['anos_privado'].mean()
    media_trocas_empresa = cluster_data['trocas_empresa'].mean()
    media_crescimento_medio_anual = cluster_data['crescimento_medio_anual'].mean()
    
    resultados.append({
        'Cluster': cluster,
        'Sexo': media_sexo,
        'trajetoria_contratos': moda['trajetoria_contratos'],
        'Dist_Tipocontr': media_dist_tipocontr,
        'CAE_Mais_Frequente': moda['cae_mais_frequente'],
        'Prof_3D_Mais_Frequente': moda['prof_3d_mais_frequente'],
        'habil1_09': media_aumentou_habil,
        'Anos_Privado': media_anos_privado,
        'Trocas_Empresa': media_trocas_empresa,
        'Crescimento_Salarial': media_crescimento_medio_anual
    })

resultados_df = pd.DataFrame(resultados)
resultados_df = resultados_df.sort_values(by='Cluster').reset_index(drop=True)

resultados_df[['Sexo', 'Dist_Tipocontr', 'habil1_09', 'Anos_Privado', 'Trocas_Empresa', 'Crescimento_Salarial']] = \
resultados_df[['Sexo', 'Dist_Tipocontr', 'habil1_09', 'Anos_Privado', 'Trocas_Empresa', 'Crescimento_Salarial']].round(2)

resultados_df

In [ ]:
# Cluster 0
cluster = df_original_clustering_4[df_original_clustering_4['cluster'] == 0]
frequencia_prof_3d = cluster['prof_3d_mais_frequente'].value_counts()
frequencia_prof_3d.head(5)

In [ ]:
# Cluster 1
cluster = df_original_clustering_4[df_original_clustering_4['cluster'] == 1]
frequencia_prof_3d = cluster['prof_3d_mais_frequente'].value_counts()
frequencia_prof_3d.head(5)

In [ ]:
# Cluster 2
cluster = df_original_clustering_4[df_original_clustering_4['cluster'] == 2]
frequencia_prof_3d = cluster['prof_3d_mais_frequente'].value_counts()
frequencia_prof_3d.head(5)

### <font color='darkred'>Clustering V</font>
#### <font color='black'>Peso na Variável aumentou_habil + Variáveis Mais Frequentes</font>

In [ ]:
variaveis_clustering_5 = [
    'sexo',
    'trajetoria_contratos',
    'dist_tipocontr',
    'crescimento_medio_anual',
    'cae_mais_frequente',
    'prof_3d_mais_frequente',
    'aumentou_habil',
    'anos_privado',
    'trocas_empresa'
]

df_original_clustering_5 = df_merged[variaveis_clustering_5].copy()
df_original_clustering_5.head()

In [ ]:
# Ajustar o impacto de 'aumentou_habil'
df_clustering_5 = df_original_clustering_5.copy()
df_clustering_5['aumentou_habil'] *= 0.5  # Reduzir o impacto da variável

In [ ]:
scaler = MinMaxScaler()
df_clustering_normalized_5 = pd.DataFrame(scaler.fit_transform(df_clustering_5), columns=df_clustering_5.columns)

In [ ]:
X = df_clustering_normalized_5.values

n_clusters_range = range(1, 11) 
inertia = []

# Calculo da inércia para cada número de clusters
for n_clusters in n_clusters_range:
    kmeans = KMeans(n_clusters=n_clusters, random_state=0)
    kmeans.fit(X)
    inertia.append(kmeans.inertia_)

plt.figure(figsize=(8, 5))
plt.plot(n_clusters_range, inertia, marker='o', color='#0D3B66')
plt.xlabel('Número de Clusters')
plt.ylabel('Inércia')
plt.title('Método do Cotovelo para Determinação do Número Ideal de Clusters')
plt.xticks(n_clusters_range)
plt.grid(True)
plt.show()

In [ ]:
X = df_clustering_normalized_5.values

n_clusters_range = range(2, 11)  
silhouette_scores = []

# Calculo do índice da silhueta para cada número de clusters
for n_clusters in n_clusters_range:
    kmeans = KMeans(n_clusters=n_clusters, random_state=0)
    cluster_labels = kmeans.fit_predict(X)
    silhouette_avg = silhouette_score(X, cluster_labels)
    silhouette_scores.append(silhouette_avg)

max_silhouette_score = max(silhouette_scores)
optimal_clusters = n_clusters_range[silhouette_scores.index(max_silhouette_score)]

print(f"O índice de silhueta mais alto é {max_silhouette_score:.2f}, obtido com {optimal_clusters} clusters.")


plt.figure(figsize=(8, 6))
plt.plot(n_clusters_range, silhouette_scores, marker='o')
plt.xlabel('Número de Clusters')
plt.ylabel('Índice da Silhueta')
plt.title('Método do Índice da Silhueta para Determinação do Número Ideal de Clusters')
plt.xticks(n_clusters_range)
plt.grid(True)
plt.text(optimal_clusters, max_silhouette_score, f"Máximo: {max_silhouette_score:.2f}", 
         horizontalalignment='left', fontsize=10, color='red')

In [ ]:
kmeans = KMeans(n_clusters=6, random_state=42)
clusters = kmeans.fit_predict(df_clustering_normalized_5)

# Adicionar a coluna de clusters ao DataFrame original
df_original_clustering_5['cluster'] = clusters
df_original_clustering_5.head()

#### Número de Trabalhadores por Cluster 

In [ ]:
n_clusters = df_original_clustering_5['cluster'].value_counts().sort_index()
n_clusters

#### Resumo das Características de Cada Cluster
- Usando a moda nas variáveis trajetoria_contratos, cae_mais_frequente e prof_3d_mais_frequente;

In [ ]:
clusters = df_original_clustering_5['cluster'].unique()

resultados = []

# Calcular moda e média para cada cluster
for cluster in clusters:
    cluster_data = df_original_clustering_5[df_original_clustering_5['cluster'] == cluster]
    
    moda = cluster_data[['trajetoria_contratos', 'cae_mais_frequente',
                         'prof_3d_mais_frequente']].mode().iloc[0]
    
    media_sexo = cluster_data['sexo'].mean()
    media_dist_tipocontr = cluster_data['dist_tipocontr'].mean()
    media_aumentou_habil = cluster_data['aumentou_habil'].mean()
    media_anos_privado = cluster_data['anos_privado'].mean()
    media_trocas_empresa = cluster_data['trocas_empresa'].mean()
    media_crescimento_medio_anual_zscore = cluster_data['crescimento_medio_anual'].mean()
    
    resultados.append({
        'Cluster': cluster,
        'Sexo': media_sexo,
        'trajetoria_contratos': moda['trajetoria_contratos'],
        'Dist_Tipocontr': media_dist_tipocontr,
        'CAE_Mais_Frequente': moda['cae_mais_frequente'],
        'Prof_3D_Mais_Frequente': moda['prof_3d_mais_frequente'],
        'Aumentou_Habil': media_aumentou_habil,
        'Anos_Privado': media_anos_privado,
        'Trocas_Empresa': media_trocas_empresa,
        'Crescimento_Salarial': media_crescimento_medio_anual_zscore
    })

resultados_df = pd.DataFrame(resultados)
resultados_df = resultados_df.sort_values(by='Cluster').reset_index(drop=True)

resultados_df[['Sexo', 'Dist_Tipocontr', 'Aumentou_Habil', 'Anos_Privado', 'Trocas_Empresa', 'Crescimento_Salarial']] = \
resultados_df[['Sexo', 'Dist_Tipocontr', 'Aumentou_Habil', 'Anos_Privado', 'Trocas_Empresa', 'Crescimento_Salarial']].round(2)

resultados_df

---
Beatriz Lapa - Tese de Mestrado